# Roadmark Dataset에서 필요한 클래스만 추출하는 코드
- 탐지하고자 하는 category_id:
    - 402: 속도제한 어린이 보호 구역
    - 403: 어린이 보호 구역
    - 426: 자전거 전용 도로
    - 412: 횡단보도
    - 432: 정차금지지대
    - 389: 주차금지
    - 390: 정차주차금지516

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import shutil

## 1) 데이터 형태 파악하기

In [ ]:
label_src_path = '/Users/myoungjikim/Downloads/087.도로_로드마크_인식을_위한_주행_영상_데이터/01.데이터/1.Training/라벨링데이터/2D_JSON'
filenames = os.listdir(label_src_path)

with open(os.path.join(label_src_path, filenames[0])) as json_data:
    result = json.load(json_data)
    pprint.pprint(result)

## 2) 데이터 추출하기

In [ ]:
# 경로 설정
label_src_path = '/Users/myoungjikim/Downloads/087.도로_로드마크_인식을_위한_주행_영상_데이터/01.데이터/1.Training/라벨링데이터/2D_JSON'
label_dst_path = '/Users/myoungjikim/Downloads/087.도로_로드마크_인식을_위한_주행_영상_데이터_preprocessed/01.데이터/1.Training/라벨링데이터/2D_JSON'
image_src_path = '/Users/myoungjikim/Downloads/087.도로_로드마크_인식을_위한_주행_영상_데이터/01.데이터/1.Training/원천데이터/TS7_02_비온전_물리_1/IMAGE'
image_dst_path = '/Users/myoungjikim/Downloads/087.도로_로드마크_인식을_위한_주행_영상_데이터_preprocessed/01.데이터/1.Training/원천데이터/TS7_02_비온전_물리_1/IMAGE'

# 탐지하고자 하는 category_id 리스트
code_to_detect = [402, 403, 426, 412, 432, 389, 390]


In [ ]:
def filter_annotations(data, categories):
    return [annotation for annotation in data['annotations'] if annotation['category_id'] in categories]

def save_json(data, path):
    with open(path, 'w') as outfile:
        json.dump(data, outfile, indent=4)
        directory, filename = os.path.split(path)
        print(f'JSON 파일 {filename}이 지정한 경로에 저장되었습니다.')

def copy_image_files(images, src_path, dst_path):
    for image in images:
        image_filename = image['filename']
        src_image_path = os.path.join(src_path, image_filename)
        dst_image_path = os.path.join(dst_path, image_filename)
        if os.path.exists(src_image_path):
            shutil.copy2(src_image_path, dst_image_path)
            print(f'이미지 파일 {image_filename}이 지정한 경로에 저장되었습니다.')
        else:
            print(f'{src_image_path} 이미지 파일이 존재하지 않습니다.')

def process_files(filenames, src_label_path, dst_label_path, src_image_path, dst_image_path, categories):
    for filename in filenames:
        json_file_path = os.path.join(src_label_path, filename)

        # JSON 파일 열기
        with open(json_file_path) as json_data:
            data = json.load(json_data)

        # 탐지하고자 하는 category_id가 아닌 annotation 제거하기
        data['annotations'] = filter_annotations(data, categories)

        # 유효한 annotation이 있는 경우에만 JSON 파일 저장 및 이미지 복사
        if data['annotations']:
            new_json_file_path = os.path.join(dst_label_path, filename)
            save_json(data, new_json_file_path)
            copy_image_files(data['images'], src_image_path, dst_image_path)
        else:
            print(f'{filename}: 탐지하고자 하는 카테고리가 없습니다.')

def main():
    filenames = os.listdir(label_src_path)  # JSON 파일 리스트
    filenames = [filename for filename in filenames if filename.endswith('.json')]

    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(label_dst_path):
        os.makedirs(label_dst_path)
    if not os.path.exists(image_dst_path):
        os.makedirs(image_dst_path)

    process_files(filenames, label_src_path, label_dst_path, image_src_path, image_dst_path, code_to_detect)

if __name__ == "__main__":
    main()


## 3) 추출한 데이터와 원본 데이터의 크기 비교

In [ ]:
import os

def get_directory_size(directory):
    total_size = 0
    # 디렉토리 내 모든 파일의 크기 합산
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            # 파일의 크기를 더함
            total_size += os.path.getsize(file_path)
    return total_size

def compare_directory_sizes(dir1, dir2):
    size1 = get_directory_size(dir1)
    size2 = get_directory_size(dir2)

    print(f"입력파일1 의 파일 크기 합: {size1} 바이트")
    print(f"입력파일2의 파일 크기 합: {size2} 바이트")


print( 'image_src_path size: ', get_directory_size(image_src_path)/1024**3,'GB' )
print( 'image_dst_path size: ', get_directory_size(image_dst_path)/1024**3,'GB' )
print('\n')
print( 'label_src_path size: ', get_directory_size(label_src_path)/1024**3,'GB' )
print( 'label_dst_path size: ', get_directory_size(label_dst_path)/1024**3,'GB' )

## 4) 바운딩 박스, 폴리라인 보기

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image

# Jupyter Notebook에서 이미지를 인라인으로 표시
%matplotlib inline

In [ ]:
def draw_annotations(image, annotations):
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    for annotation in annotations:
        if 'bbox' in annotation:
            # 바운딩 박스 그리기
            x, y, w, h = annotation['bbox']
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='g', facecolor='none')
            ax.add_patch(rect)
        if 'polyline' in annotation:
            # 폴리라인 그리기
            points = annotation['polyline']
            # Reshape 1D polyline array into pairs of (x, y) coordinates
            points = np.array(points).reshape(-1, 2)
            polyline = Line2D(points[:, 0], points[:, 1], linewidth=2, color='b')
            ax.add_line(polyline)

    return fig, ax

def process_json_file(json_file_path, image_src_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # 이미지 파일 이름 얻기
    image_filename = data['images'][0]['filename']
    image_path = os.path.join(image_src_path, image_filename)


    # 이미지 읽기
    image = Image.open(image_path)

    # 어노테이션 그리기
    fig, ax = draw_annotations(np.array(image), data['annotations'])

    # Jupyter Notebook에서 이미지를 인라인으로 표시
    plt.show()

def main():
    label_filenames = os.listdir(label_dst_path)
    #                                            label_filenames[여기바꿔서] 이미지변경 가능
    json_file_path = os.path.join(label_dst_path,label_filenames[120])  # JSON 파일 경로
    image_src_path = image_dst_path # 원본 이미지 폴더 경로

    process_json_file(json_file_path, image_src_path)

if __name__ == "__main__":
    main()
